In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    Settings,
    get_response_synthesizer)
from llama_index.core.query_engine import RetrieverQueryEngine, TransformQueryEngine
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import TextNode, MetadataMode
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
import qdrant_client
import logging

In [ ]:
client = qdrant_client.QdrantClient(
    url="https://bacf7390-f1c5-4b61-a9f2-4443989f3295.us-east4-0.gcp.cloud.qdrant.io:6333/",
    api_key="lG6om4PK4PRLcGruuTdGdIggIj9KUMi2ZPhJ6DXlHAKxIoDo5EDebw")
vector_store = QdrantVectorStore(client=client, collection_name="R18_BTech_AIML_Syllabus_local_pc")

In [ ]:
Settings.embed_model = OllamaEmbedding(model_name='mxbai-embed-large', base_url='http://localhost:11434')
Settings.llm = Ollama(model="llama3", base_url='http://localhost:11434')
Settings.system_prompt = "Ask me anything about R18 BTech AIML Syllabus"

In [ ]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='R18_BTech_AIML_Syllabus_local_pc'), CollectionDescription(name='R18_BTech_AIML_Syllabus')])

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_vector_store(vector_store=vector_store, storage_context=storage_context)

In [ ]:
vector_retriever = VectorIndexRetriever(index=index, similarity_top_k=5)
response_synthesizer = get_response_synthesizer()

vector_query_engine = RetrieverQueryEngine(
    retriever=vector_retriever,
    response_synthesizer=response_synthesizer,
)

hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(vector_query_engine, hyde)

In [ ]:
response = hyde_query_engine.query(
    str_or_query_bundle="what are the topics of CRYPTOGRAPHY AND NETWORK SECURITY in 3rd year and 2nd semister syllabus.get me an overview on every topic.",)
print(response)